# Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from utils import TrainTestSequenceDataset, PredictSequenceDataset, smape, process_for_train_test, features, targets, process_for_predict
from models import LSTM
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import ipywidgets as widgets
import os
from spaceopt import SpaceOpt
from time import strptime
import warnings
warnings.filterwarnings('ignore')

# Loading data

In [2]:
data = pd.read_csv('train.csv', parse_dates=['epoch'],
                   infer_datetime_format=True)
# test_sat_id = np.unique(pd.read_csv('train.csv')['sat_id'].values)

# Data processing

In [3]:
sat_datas_train, sat_datas_test = process_for_train_test(data)

In [4]:
sat_datas_train[0].head()

,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,dt,dx_sim,dy_sim,dz_sim
0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-1.403236,0.842595,-1.424594,-0.970349,-1.621059,-0.935724,0.037341,-0.969679,-1.625401,-0.935026
1,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-1.577905,0.372368,-1.589195,-0.327658,-1.818828,-0.287952,0.037341,-0.326737,-1.823455,-0.286974
2,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-1.579573,-0.110396,-1.580288,0.288169,-1.723948,0.326491,0.037351,0.289331,-1.728439,0.327735
3,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-1.434499,-0.539012,-1.425626,0.753891,-1.438247,0.786453,0.037341,0.755235,-1.442325,0.787896
4,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-1.187622,-0.878068,-1.170987,1.048664,-1.075547,1.073571,0.037351,1.050123,-1.079101,1.075138


In [5]:
sat_datas_test[0].head()

,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim,dt,dx_sim,dy_sim,dz_sim
718,3478.786221,29819.020220,7062.855377,-1.525112,0.075176,-3.517007,-0.638027,1.449782,-0.685976,-1.674371,-0.564877,-1.652014,0.037351,-1.673977,-0.567694,-1.651626
719,-910.353216,28313.233297,-3003.729365,-1.577225,-1.188958,-3.596677,-1.065314,1.218266,-1.104655,-1.469152,-1.152524,-1.426653,0.037341,-1.468677,-1.156189,-1.426167
720,-5156.221079,23031.794108,-12619.286635,-1.409264,-2.575675,-3.165172,-1.400004,0.840016,-1.425764,-0.990359,-1.622241,-0.932227,0.037351,-0.989697,-1.626585,-0.931528
721,-8562.860529,14092.099579,-20183.438335,-0.981024,-3.727017,-2.141273,-1.580052,0.369645,-1.589236,-0.347450,-1.818557,-0.283452,0.037341,-0.346537,-1.823184,-0.282472
722,-10493.335805,2727.541857,-24267.597152,-0.382391,-4.259435,-0.744472,-1.586861,-0.112854,-1.579272,0.269869,-1.722433,0.329579,0.037351,0.271024,-1.726922,0.330825


In [6]:
device = torch.device('cuda')

# Model

In [7]:
seq_len = 3
hidden_dim = 30
num_layers = 2
model = LSTM(hidden_dim=hidden_dim, seq_len=seq_len, num_layers=num_layers, input_dim=10)
model.train()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001, eps=1e-4)
criterion = smape
n = 10
model.to(device)

LSTM(
  (lstm): LSTM(10, 30, num_layers=2, batch_first=True, dropout=0.1)
  (dropout): Dropout(p=0.1, inplace=False)
  (activation): ReLU()
  (linear): Linear(in_features=30, out_features=6, bias=True)
)

# Train on n satellite

In [ ]:
model.train()
loss_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Loss', value=0)  # jupyter widget
display(loss_widget)
train_data = sat_datas_train[n]
x_train = train_data[features]
y_train = train_data[targets]
train_dataset = TrainTestSequenceDataset(x_train, y_train, seq_len=model.seq_len)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
for epoch in tqdm(range(10)):
    for seq_train_x, seq_train_y in train_dataloader:
        seq_train_x = seq_train_x
        seq_train_y = seq_train_y.cuda()
        model.zero_grad()  # refresh gradients
        model.init_hidden_cell()
        model.hidden_cell = (model.hidden_cell[0], model.hidden_cell[1])
        
        predictions = model(seq_train_x).squeeze(0)
        predictions.to(device)
        loss = criterion(predictions, seq_train_y)
        loss_widget.value = loss.mean()
        loss.mean().backward()  # compute gradients
        optimizer.step()  # update network parameters

FloatProgress(value=0.0, description='Loss', max=1.0)

In [ ]:
model.eval()
score_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Score', value=0)  # jupyter widget
display(score_widget)
test_data = sat_datas_test[n]
x_test = test_data[features]
y_test = test_data[targets]
test_dataset = TrainTestSequenceDataset(x_test, y_test, seq_len=model.seq_len)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
loss_sum = 0
i = 0
model = model.to(device)
for seq_test_x, seq_test_y in tqdm(test_dataloader):
    with torch.no_grad():
        seq_test_x = seq_test_x.cuda()
        seq_test_y = seq_test_y.cuda()
        model.init_hidden_cell()
        model.hidden_cell = (model.hidden_cell[0].cuda(), model.hidden_cell[1].cuda())
        predictions = model(seq_test_x)
        predictions = predictions.cuda()
        loss = criterion(predictions, seq_test_y).mean()
        loss_sum += loss
        i += 1
        score_widget.value = 1 - loss_sum / i
score =  (1 - loss_sum / i).item()
print(score)

In [ ]:
search_space = {
    'lr': [0.001, 0.01, 0.1, 1.],
    'eps': [1e-8, 1e-4, 1e-2, 1e0],
    'seq_len': [10, 20, 30, 40],
    'hidden_dim': [20, 30, 50],
    'epoch': [10],
    'num_layers': [1, 2, 3, 4]
}

# Models evaluation

In [ ]:
def evaluate_new(spoint, sat_id):
    seq_len = spoint['seq_len']
    hidden_dim = spoint['hidden_dim']
    num_layers = spoint['num_layers']
    ep = spoint['epoch']
    lr = spoint['lr']
    eps = spoint['eps']
    
    train_data = sat_datas_train[sat_id]
    if seq_len > int(len(train_data) / 2 - 1):
        seq_len = int(len(train_data) / 2 - 1)
    x_train = train_data[features]
    x_train = x_train.drop(['ro_sim', 'theta_sim', 'fi_sim', 'dro/dt_sim', 'dtheta/dt_sim', 'dfi/dt_sim',
                 'dro_sim', 'dtheta_sim', 'dfi_sim'], axis=1)
    y_train = train_data[targets]
    
    model = LSTM(hidden_dim=hidden_dim, seq_len=seq_len, num_layers=num_layers,
                 input_dim=x_train.shape[1])
    model.train()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr, eps=eps)
    criterion = smape
    model.train()
    loss_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Loss', value=0)  # jupyter widget
    display(loss_widget)
    
    train_dataset = TrainTestSequenceDataset(x_train, y_train, seq_len=model.seq_len)
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    for epoch in tqdm(range(10)):
        for seq_train_x, seq_train_y in train_dataloader:
            model.zero_grad()  # refresh gradients
            model.init_hidden_cell()
            predictions = model(seq_train_x)
            loss = criterion(predictions, seq_train_y)
            loss_widget.value = loss.mean()
            loss.mean().backward()  # compute gradients
            optimizer.step()  # update network parameters

    model.eval()
    score_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Score', value=0)  # jupyter widget
    display(score_widget)
    test_data = sat_datas_test[sat_id]
    x_test = test_data[features]
    y_test = test_data[targets]
    test_dataset = TrainTestSequenceDataset(x_test, y_test, seq_len=model.seq_len)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    loss_sum = 0
    i = 0

    for seq_test_x, seq_test_y in tqdm(test_dataloader):
        with torch.no_grad():
            model.init_hidden_cell()
            predictions = model(seq_test_x)
            loss = criterion(predictions, seq_test_y).mean()
            loss_sum += loss
            i += 1
            score_widget.value = 1 - loss_sum / i
    score =  (1 - loss_sum / i).item()

    return score, model

In [ ]:
best_params = {'lr': 0.02, 'eps': 1e-2, 'seq_len': 5, # seq_len 5 hidden_dim 30
               'hidden_dim': 20, 'epoch': 10, 'num_layers': 2}

In [ ]:
score, model = evaluate_new(spoint=spoint, sat_id=599)

In [ ]:
score

In [ ]:
torch.save(score, f'models//{sat_id}//score')
torch.save(model, f'models//{sat_id}//model.pt')

In [ ]:
results = {}
for sat_id in test_sat_id:
    clear_output()
    print(f'Satellite id: {sat_id}')
    print(results)
    spoint = best_params
    
    score, model= evaluate_new(spoint=spoint, sat_id=sat_id)
    spoint['score'] = score
    if not os.path.exists(f'models//{sat_id}'):
        os.makedirs(f'models//{sat_id}')
    results[str(sat_id)] = score
    torch.save(score, f'models//{sat_id}//score')
    torch.save(model, f'models//{sat_id}//model.pt')

# Models factory

In [ ]:
best_params = {'lr': 0.02, 'eps': 1e-2, 'seq_len': 20,
               'hidden_dim': 100, 'epoch': 30, 'num_layers': 2}

In [ ]:
data = pd.read_csv('train.csv', parse_dates=['epoch'])
sat_datas_train = process_for_train(data)

In [ ]:
sat_datas_train[0]

In [ ]:
def train_new(spoint, sat_id):
    seq_len = spoint['seq_len']
    hidden_dim = spoint['hidden_dim']
    num_layers = spoint['num_layers']
    ep = spoint['epoch']
    lr = spoint['lr']
    eps = spoint['eps']
    
    train_data = sat_datas_train[sat_id]
    if seq_len > int(len(train_data) / 2 - 1):
        seq_len = int(len(train_data) / 2 - 1)
    x_train = train_data[features]
    y_train = train_data[targets]
    
    model = LSTM(hidden_dim=hidden_dim, seq_len=seq_len, num_layers=num_layers,
                 input_dim=x_train.shape[1])
    model.train()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr, eps=eps)
    criterion = smape
    model.train()
    loss_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Loss', value=0)  # jupyter widget
    display(loss_widget)
    train_data = sat_datas_train[sat_id]
    x_train = train_data[features]
    y_train = train_data[targets]
    train_dataset = TrainTestSequenceDataset(x_train, y_train, seq_len=model.seq_len)
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    for epoch in tqdm(range(ep)):
        for seq_train_x, seq_train_y in train_dataloader:
            model.zero_grad()  # refresh gradients
            model.init_hidden_cell()
            predictions = model(seq_train_x)
            loss = criterion(predictions, seq_train_y)
            loss_widget.value = loss.mean()
            loss.mean().backward()  # compute gradients
            optimizer.step()  # update network parameters

    model.eval()

    return model

In [ ]:
for sat_id in range(600):
    clear_output()
    print(f'Satellite id: {sat_id}')
    spoint = best_params
    
    model= train_new(spoint=spoint, sat_id=sat_id)
    if not os.path.exists(f'models//{sat_id}'):
        os.makedirs(f'models//{sat_id}')
    
    torch.save(model, f'models//{sat_id}//model.pt')

In [ ]:
sat_id = 599
model= train_new(spoint=best_params, sat_id=sat_id)
if not os.path.exists(f'models//{sat_id}'):
    os.makedirs(f'models//{sat_id}')

torch.save(model, f'models//{sat_id}//model.pt')